# References
* [hf create tokenizer](https://m.youtube.com/watch?v=jee78mdPL6A)
* [hf create dataset](https://m.youtube.com/watch?v=enObIMzyaE4)
* [train gpt2 from scratch](https://m.youtube.com/watch?v=98jROp7Ij9A&t=5s&pp=2AEFkAIB)
* [dataset: reuters21578](https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.html)

In [ ]:
!pip install datasets --quiet transformers torch huggingface evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!wget https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.tar.gz

--2024-10-30 09:07:32--  https://kdd.ics.uci.edu/databases/reuters21578/reuters21578.tar.gz
Resolving kdd.ics.uci.edu (kdd.ics.uci.edu)... 128.195.1.86
Connecting to kdd.ics.uci.edu (kdd.ics.uci.edu)|128.195.1.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8150596 (7.8M) [application/x-gzip]
Saving to: ‘reuters21578.tar.gz’

reuters21578.tar.gz 100%[===================>]   7.77M  11.9MB/s    in 0.7s    

2024-10-30 09:07:33 (11.9 MB/s) - ‘reuters21578.tar.gz’ saved [8150596/8150596]



In [ ]:
# # unzip
# !tar -xzvf reuters21578.tar.gz

In [ ]:
from datasets import load_dataset
dataset = load_dataset('ingeniumacademy/reuters_articles')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/698 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.13M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/827k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17262 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2158 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2158 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'body'],
        num_rows: 17262
    })
    validation: Dataset({
        features: ['title', 'body'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['title', 'body'],
        num_rows: 2158
    })
})

In [ ]:
# tokenizer
def create_full_article_col(example):
    return {'full_article':f"TITLE:{example['title']}\n\nBODY:{example['body']}"}

dataset = dataset.map(create_full_article_col)
dataset

Map:   0%|          | 0/17262 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'full_article'],
        num_rows: 17262
    })
    validation: Dataset({
        features: ['title', 'body', 'full_article'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['title', 'body', 'full_article'],
        num_rows: 2158
    })
})

In [ ]:
dataset['train'][0]['full_article']

'TITLE:BAHIA COCOA REVIEW\n\nBODY:Showers continued throughout the week in\nthe Bahia cocoa zone, alleviating the drought since early\nJanuary and improving prospects for the coming temporao,\nalthough normal humidity levels have not been restored,\nComissaria Smith said in its weekly review.\n    The dry period means the temporao will be late this year.\n    Arrivals for the week ended February 22 were 155,221 bags\nof 60 kilos making a cumulative total for the season of 5.93\nmln against 5.81 at the same stage last year. Again it seems\nthat cocoa delivered earlier on consignment was included in the\narrivals figures.\n    Comissaria Smith said there is still some doubt as to how\nmuch old crop cocoa is still available as harvesting has\npractically come to an end. With total Bahia crop estimates\naround 6.4 mln bags and sales standing at almost 6.2 mln there\nare a few hundred thousand bags still in the hands of farmers,\nmiddlemen, exporters and processors.\n    There are doubts as

# Training our own tokenizer

In [ ]:
# Create a batched dataset for training. creates an iterator for later usage when training  tokenizer
training_corpus = (
    dataset["train"][i:i+10000]["full_article"]
    for i in range(0, len(dataset["train"]), 10000)
)

In [ ]:
from transformers import AutoTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
example = dataset['test']['full_article'][0]
example

'TITLE:AMEX SAYS SHORT INTEREST UP 123,002 SHARES FOR MONTH\nENDED JUNE 15\n\n\nBODY:'

In [ ]:
old_tokenizer.tokenize(example) # TITLE is splitted into tokens: 'TIT' 'LE'

['TIT',
 'LE',
 ':',
 'AM',
 'EX',
 'ĠSAY',
 'S',
 'ĠSH',
 'ORT',
 'ĠINTER',
 'EST',
 'ĠUP',
 'Ġ123',
 ',',
 '002',
 'ĠSHARES',
 'ĠFOR',
 'ĠMON',
 'TH',
 'Ċ',
 'ENDED',
 'ĠJ',
 'UNE',
 'Ġ15',
 'ĊĊ',
 'Ċ',
 'B',
 'ODY',
 ':']

In [ ]:
# new tokenizer
new_tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000) # Vocab size of 5200

In [ ]:
new_tokenizer.tokenize(example) # TITLE is single token

['TITLE',
 ':',
 'AMEX',
 'ĠSAYS',
 'ĠSHORT',
 'ĠINTEREST',
 'ĠUP',
 'Ġ123',
 ',',
 '002',
 'ĠSHARES',
 'ĠFOR',
 'ĠMONTH',
 'Ċ',
 'ENDED',
 'ĠJUNE',
 'Ġ15',
 'ĊĊ',
 'Ċ',
 'BODY',
 ':']

In [ ]:
# # To upload the tokenizer
# from huggingface import notebook_login
# notebook_login()
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
new_tokenizer.push_to_hub('gpt2-reuters-tokenizer')

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6721f867-662647dd257202d453648549;0caadf6a-4c14-4755-8596-0aa377636c4c)

Invalid username or password.

# GPT-2 Model

## use our tokenizer

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("ingeniumacademy/gpt2-reuters-tokenizer")
tokenizer.tokenize("Hello, my dog is cute")

tokenizer_config.json:   0%|          | 0.00/440 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/819k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/465k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.17M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

['H', 'ello', ',', 'Ġmy', 'Ġdog', 'Ġis', 'Ġcut', 'e']

In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in 

In [ ]:
from datasets import load_dataset
dataset = load_dataset('ingeniumacademy/reuters_articles')
dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'body'],
        num_rows: 17262
    })
    validation: Dataset({
        features: ['title', 'body'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['title', 'body'],
        num_rows: 2158
    })
})

In [ ]:
# tokenizer
def create_full_article_col(example):
    # merge title and body
    return {'full_article':f"TITLE:{example['title']}\n\nBODY:{example['body']}"}

dataset = dataset.map(create_full_article_col)
dataset

Map:   0%|          | 0/17262 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'body', 'full_article'],
        num_rows: 17262
    })
    validation: Dataset({
        features: ['title', 'body', 'full_article'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['title', 'body', 'full_article'],
        num_rows: 2158
    })
})

In [ ]:
# Tokenize the dataset for context length = 512 (truncate longer text)
CONTEXT_LENGTH = 512

def tokenize(element):
    outputs = tokenizer(
        element['full_article'],
        truncation=True,
        max_length=CONTEXT_LENGTH,
        return_overflowing_tokens=False # This line
    )
    return outputs


tokenized_datasets = dataset.map(tokenize, batched=True, remove_columns=dataset['train'].column_names)
tokenized_datasets

Map:   0%|          | 0/17262 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 17262
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2158
    })
})

## Preparing model for training

In [ ]:
from transformers import AutoTokenizer, AutoConfig, GPT2LMHeadModel

config =AutoConfig.from_pretrained(
    "gpt2", # Instead of "gpt2", try "gpt2-medium" or "gpt2-small"
    vocab_size=len(tokenizer),
    n_ctx=CONTEXT_LENGTH,
    bos_token_id=tokenizer.bos_token_id,  # 0 : <|endoftext|>
    eos_token_id=tokenizer.eos_token_id,  # 0 : <|endoftext|>
)
config

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 0,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_version": "4.44.2",
  "use_cache": true,
  "vocab_size": 52000
}

In [ ]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 125.8M parameters


In [ ]:
from transformers import DataCollatorForLanguageModeling
# -------------------
# Log in to wandb
# wandb.login()
# -------------------

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 17262
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2158
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2158
    })
})

In [ ]:
# training the model
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    # crashed after using up all available RAM:
    # per_device_train_batch_size=16, # Try smaller values if needed
    #  gradient_checkpointing=True,

    output_dir="./gpt2-reuters",  # local directory
    hub_model_id = "Aananda-giri/gpt2-reuters",  # identifier on the hub
    evaluation_strategy="epoch",
    auto_find_batch_size=True,
    num_train_epochs=2,
    gradient_accumulation_steps= 8, # try 16 if it keeps crashing
    weight_decay=.1,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    fp16=True,  # Floating point 16 (default 32)
    push_to_hub=False,
    logging_steps=10,
    report_to="wandb"  # `none` disables wandb and all other integrations

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer =Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset = tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,5.158400,5.274678
1,4.789900,4.997947


TrainOutput(global_step=538, training_loss=5.407628920884823, metrics={'train_runtime': 1431.5376, 'train_samples_per_second': 24.117, 'train_steps_per_second': 0.376, 'total_flos': 6943268256768000.0, 'train_loss': 5.407628920884823, 'epoch': 1.9944392956441148})

In [ ]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Aananda-giri/gpt2-reuters/commit/60d90eec665d287a2b692968c850993fa97fff7d', commit_message='End of training', commit_description='', oid='60d90eec665d287a2b692968c850993fa97fff7d', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# -------------------
# Finish the wandb run after training
wandb.finish()
# -------------------

# using our model in pipeline

In [ ]:
import torch
from transformers import pipeline

pipe = pipeline(
    "text-generation", model="Aananda-giri/gpt2-reuters"
    )

config.json:   0%|          | 0.00/898 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/503M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/814k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
sample = dataset['test'][2]
sample

{'title': 'CHEFS <CHEF.O> COMPLETES PRIVATE SALE',
 'body': "Chefs International\nInc said it completed a private sale of nine mln units of its\nsecurities for 20 cts per unit for a total price of 1,800,000\ndlrs.\n    The company said each unit consisted of one share of Chef's\ncommon stock and one three-year warrant exerciseable to\npurchase one share of Chef's stock at 25 cents.\n    The company also said Robert E. Brennan purchased 8,250,000\nof the units.\n   \n Reuter\n\x03",
 'full_article': "TITLE:CHEFS <CHEF.O> COMPLETES PRIVATE SALE\n\nBODY:Chefs International\nInc said it completed a private sale of nine mln units of its\nsecurities for 20 cts per unit for a total price of 1,800,000\ndlrs.\n    The company said each unit consisted of one share of Chef's\ncommon stock and one three-year warrant exerciseable to\npurchase one share of Chef's stock at 25 cents.\n    The company also said Robert E. Brennan purchased 8,250,000\nof the units.\n   \n Reuter\n\x03"}

In [ ]:
prompt = f'TITLE:{sample["title"]}\n\nBODY:'
pipe(prompt, max_new_tokens=128)

[{'generated_text': 'TITLE:CHEFS <CHEF.O> COMPLETES PRIVATE SALE\n\nBODY:The European Co of\nInc said, to acquire the proposed sales of\nreceived the company.\n    It said they will offer in the stock and\n Reuter\n\x03 15.2 cts a company,000 shares with the same shares of the\n\x03 to purchase from the stock to the sale of the\nfor a year.\n    "The agreement not not put up March 27,000 dlrs.\n    The company said that would not also gave not\nwill also announced by the annual debt for\nyear of the year. and\n Reuter\n\x03 five pct for a share by the offer, and 1.1 mln dlrs a\n\x03 half'}]

## keep training pre-trained model

In [ ]:
# load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

pre_tokenizers = AutoTokenizer.from_pretrained("ingeniumacademy/reuters-gpt2-text-gen)
pre_model = AutoModelForCausalLM.from_pretrained("ingeniumacademy/reuters-gpt2-text-gen)